# Solving Logistic Problems with ORS and or-tools

The logistics of delivering physical items as efficiently as possible has always been of huge interest for businesses. With the rise of online shopping more and more packages are delivered every day to even private consumers. Therefore, making delivery services more effective is a lucrative field for companies all over the world.

Here, we want to showcase an easy solution to generic logistic problems using [**or-tools**](https://github.com/google/or-tools) and the [**openrouteservice API**](https://openrouteservice.org/). The first step is to import the required packages. 

In [1]:
from IPython.core.display import display, HTML
import openrouteservice
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import folium
from shapely import wkt, geometry
import numpy as np 
import pandas as pd

# Setting up the problem

Let say we a are small delivery company in Heidelberg and we have 21 customers asking us to deliver their package orders.  Of course every client is ordering something different and hence the package sizes vary. We quantify the package size by a demand number. A higher demand number means a bigger package needs to be delivered. As a small company, we only have 4 delivery trucks and each of them can only transport a maximum demand of 100. Our goal is to deliver the packages as fast as possible.  

In [2]:
# address names and demand of the 21 customers + our company (with demand 0)
CVRP = np.array([[  'Quinckestraße 49',             0   ],
                 [  'Bergfriedhof',                 24  ],
                 [  'Gertrude-von-Ubisch-Straße',   6   ],
                 [  'Speyerer Straße',              20  ],
                 [  'Klostergasse 6',               9   ],
                 [  'Feuerbachstraße 26',           19  ],
                 [  'A5',                           11  ],
                 [  'Mittlerer Gaisbergweg 11',     22  ],
                 [  'Eisenhower Street',            18  ],
                 [  'Czernyring 10',                6   ],
                 [  'Handelsstraße 1',              23  ],                
                 [  'Traitteurweg',                 21  ],
                 [  'Baumschulenweg',               3   ],
                 [  'Schleifweg 44',                28  ],
                 [  'Oelgasse 1a',                  13  ],
                 [  'Rudolf-Diesel-Straße 20',      4   ],
                 [  'Yorckstraße',                  26  ],
                 [  'Am Taubenfeld 35',             25  ],
                 [  'Ringstraße 19a',               2   ],
                 [  'Wieblinger Weg 100a',          4   ],
                 [  'Alte Glockengießerei',         15  ],
                 [  'Hermann-Schück-Weg 1',         11  ]])

# problem data definition 
names = CVRP[:, 0]
demands = CVRP[:, 1].astype(int) 
depot = 0    # The depot is the start and end point of each route aka our company domicile
num_vehicles = 4
capacity = 100

# centroid of Heidelberg
centroid = geometry.Point(8.694361786680039, 49.40548948222444)

We know the adress and the demand of our customers. First, let us figure out where our custormers are located. The [**geocoding API**](https://openrouteservice.org/documentation/#/reference/geocode) is a suitable tool to accomplish this. 

In [5]:
# Prepare the client
api_key = '58d904a497c67e00015b45fc9298e8d961e64b48b066a43e51d39887'
client = openrouteservice.Client(key=api_key)

# Request coordinates from geocoding API
locations = []
for name in names:
    response = client.pelias_search(text=name, circle_point=(centroid.x, centroid.y), circle_radius=20)
    location = response['features'][0]['geometry']['coordinates']
    locations.append(location)
    
# problem data definition 
locations = np.array(locations)
num_locations = len(locations)

# add coordinates to CVRP
CVRP = np.concatenate((CVRP, locations), axis=1)

# create dataframe
df_CVRP = pd.DataFrame(CVRP, columns=['address', 'demand', 'longitude', 'latitude'])
df_CVRP.columns.name = 'node number'
display(df_CVRP)

node number,address,demand,longitude,latitude
0,Quinckestraße 49,0,8.683448,49.416961
1,Bergfriedhof,24,8.691616,49.394692
2,Gertrude-von-Ubisch-Straße,6,8.662789,49.37446
3,Speyerer Straße,20,8.651058,49.379836
4,Klostergasse 6,9,8.696694,49.295029
5,Feuerbachstraße 26,19,8.684574,49.393819
6,A5,11,8.457435,49.487161
7,Mittlerer Gaisbergweg 11,22,8.693395,49.401669
8,Eisenhower Street,18,8.685109,49.387783
9,Czernyring 10,6,8.670162,49.406377


A quick plot of the coordinates will give some insight into where all the packages need to be delivered to:

In [6]:
# create folium map  
plot_map = folium.Map(tiles='stamenterrain',location=(centroid.y, centroid.x), zoom_start=12) 

# add company location to map
popup = '<strong>Our Company (Depot):</strong> {}'.format(names[0])
icon = folium.Icon(color='blue',icon_color='black', icon='home' , prefix='fa')
folium.Marker(locations[0, ::-1], popup=popup, icon=icon).add_to(plot_map) 

# add customer location to map
for counter, point in enumerate(locations[1:]):
    popup = '<strong>Node {}:</strong> {} <br>demand: {}'.format(counter+1, names[counter+1], demands[counter+1])
    icon = folium.Icon(color='green', icon_color='black', icon='child', prefix='fa')
    folium.Marker(point[::-1], popup=popup, icon=icon).add_to(plot_map)
    
# show map
display(plot_map)

The blue marker shows the location of our company and the green markers are the locations of the customers. Clicking on the markers will show the node number, address and demand. For a detailed walkthrough of how the CVRP and Heidelberg polygon was created, see [**here**](https://github.com/Xenovortex/Setting-up-a-CVRP).

# Solving the logistic problem

First, we use the [**matrix API**](https://openrouteservice.org/documentation/#/reference/matrix/matrix), which will give us a symmetric duration matrix for a list of locations/coordinates, where every location is paired with each other. The values of the matrix are given in seconds. 

In [7]:
response = client.distance_matrix(locations=locations.tolist(), metrics=['duration'])
duration_matrix = np.array(response['durations']).astype(int)
df_duration_matrix = pd.DataFrame(duration_matrix)
display(df_duration_matrix.head())

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,430,635,520,1121,392,1118,423,404,248,...,338,617,662,1620,1274,643,1601,487,296,659
1,438,0,530,436,887,159,1152,185,169,280,...,254,333,370,1387,1307,677,1513,521,230,375
2,585,492,0,177,664,438,1157,510,400,416,...,300,366,496,1164,1313,781,1260,530,377,408
3,468,375,227,0,687,321,1060,393,333,299,...,183,387,517,1187,1215,664,1163,432,260,429
4,1106,860,699,653,0,821,1614,965,732,938,...,821,697,827,607,1713,1272,821,1005,899,739


As we can see the openrouteservice API returns a symmetric duration matrix with the diagonal being zeros, since the duration to get from every node to itself is zero. 

The SetArcCostEvaluatorOfAllVehicles() and AddDimension() method of the or-tool package requires a callable object. Therefore we have to wrap the duration matrix and the demands within a class with suitable methods that can be called by named methods. 

In [8]:
class matrix(object): 
    """Creates callback to return duration between points."""
    def __init__(self, matrix):
        """Initializes the duration matrix."""
        self._matrix = matrix

    def Duration(self, from_node, to_node):
        """Returns the duration between the two nodes"""
        return self._matrix[from_node][to_node]

class demand(object): 
    """Creates callback to get demands at each location."""
    def __init__(self, demands):
        """Initializes the demand array."""
        self._demands = demands

    def Demand(self, from_node, to_node):
        """Returns the demand of the current node"""
        del to_node
        return self._demands[from_node]

In the next step, we use the or-tool to solve the problem with our obtained duration matrix and demand modified with suitable callbacks:

In [24]:
# Create Routing Model
routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)

# Define weight of each edge
duration_evaluator = matrix(duration_matrix).Duration
routing.SetArcCostEvaluatorOfAllVehicles(duration_evaluator)

# Add Capacity constraint
demand_evaluator = demand(demands).Demand
routing.AddDimension(demand_evaluator, 0, capacity, True, "Capacity") 

# Setting first solution heuristic (cheapest addition)
search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem
assignment = routing.SolveWithParameters(search_parameters)

# Print the solution
total_dur = 0
routes = []
for vehicle_id in range(num_vehicles):
    index = routing.Start(vehicle_id)
    plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
    route_dur = 0
    route_load = 0
    temp_route = []
    while not routing.IsEnd(index):
        node_index = routing.IndexToNode(index)
        next_node_index = routing.IndexToNode(assignment.Value(routing.NextVar(index)))
        route_dur += matrix(duration_matrix).Duration(node_index, next_node_index)
        route_load += demands[node_index]
        temp_route.append(node_index)
        plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
        index = assignment.Value(routing.NextVar(index))
    node_index = routing.IndexToNode(index)
    total_dur += route_dur
    temp_route.append(node_index)
    routes.append(temp_route) 
    plan_output += ' {0} Load({1})\n'.format(node_index, route_load)
    plan_output += 'Duration of the route: {0:.2f}min\n'.format(route_dur / 60)
    plan_output += 'Load of the route: {0}\n'.format(route_load)
    print(plan_output)

Route for vehicle 0:
 0 Load(0) ->  17 Load(25) ->  10 Load(48) ->  19 Load(52) ->  0 Load(52)
Duration of the route: 33.85min
Load of the route: 52

Route for vehicle 1:
 0 Load(0) ->  6 Load(11) ->  16 Load(37) ->  18 Load(39) ->  15 Load(43) ->  4 Load(52) ->  14 Load(65) ->  8 Load(83) ->  0 Load(83)
Duration of the route: 102.80min
Load of the route: 83

Route for vehicle 2:
 0 Load(0) ->  7 Load(22) ->  1 Load(46) ->  5 Load(65) ->  20 Load(80) ->  0 Load(80)
Duration of the route: 20.50min
Load of the route: 80

Route for vehicle 3:
 0 Load(0) ->  9 Load(6) ->  12 Load(9) ->  3 Load(29) ->  2 Load(35) ->  11 Load(56) ->  13 Load(84) ->  21 Load(95) ->  0 Load(95)
Duration of the route: 30.78min
Load of the route: 95



In [15]:
a = [1, 2, 3]
b = tuple(a)
print(b)

(1, 2, 3)


The longest route only takes 102.80min, which means that we can deliver all packages under two hours.

For some visualisation, we can plot the 4 routes with folium using the openrouteservice [**direction API**](https://openrouteservice.org/documentation/#/reference/directions) with the following color assignment:
- Route for vehicle 0: red
- Route for vehicle 1: green
- Route for vehicle 2: blue
- Route for vehicle 3: black